# Create a Map for radiation numbers

In [2]:
import pandas as pd
import folium

# Laden Sie die CSV-Datei
datei_pfad = "Data\\Globale Strahlung.csv"
daten = pd.read_csv(datei_pfad)

# Initialisieren der Karte - wählen Sie einen zentralen Punkt in Schleswig-Holstein
karte = folium.Map(location=[54.3233, 10.1228], zoom_start=8)

# Durch die Daten iterieren und Marker hinzufügen
for index, zeile in daten.iterrows():
    ort = zeile['Ort']
    latitude = zeile['Latitude']
    longitude = zeile['Longitude']
    
    # Hinzufügen eines Markers zur Karte
    folium.Marker(
        location=[latitude, longitude],
        popup=ort,
    ).add_to(karte)

# Karte speichern
karte.save("Datenverfügbarkeit Globale Strahlung.html")

Wind speed


In [5]:
import os
os.getcwd()

'c:\\Users\\M97947\\OneDrive - E.ON\\Dokumente\\Thesis\\Code\\fca_leistungsbaender\\exploratory'

In [7]:
import pandas as pd
import folium
import math

# Excel laden (ggf. Sheetname anpassen)
datei_pfad = r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\Data\Feature Verfügbarkeit\Windgeschwindigkeit.xlsx"
# sheet_name weglassen = erstes Blatt, sonst z.B.: sheet_name="Tabelle1"
df = pd.read_excel(datei_pfad)  # benötigt: pip install openpyxl

# Karte initialisieren (Schleswig-Holstein)
karte = folium.Map(location=[54.3233, 10.1228], zoom_start=8)

def safe_str(x):
    if isinstance(x, float) and math.isnan(x):
        return ""
    return str(x) if x is not None else ""

# Marker setzen
for idx, row in df.iterrows():
    ort = safe_str(row.get('Ort'))
    lat = row.get('Latitude')
    lon = row.get('Longitude')
    dab_id = safe_str(row.get('DAB_ID'))
    datapoint_id = safe_str(row.get('Datapoint_ID'))

    if pd.notna(lat) and pd.notna(lon):
        popup_html = f"""
        <div style="font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial; font-size:14px; line-height:1.3; min-width:260px;">
          <div style="font-weight:600; margin-bottom:6px;">{ort}</div>

          <div style="margin-bottom:6px;">
            <div style="font-size:12px; color:#555;">DAB_ID</div>
            <div style="display:flex; gap:6px; align-items:center;">
              <input id="dab{idx}" type="text" value="{dab_id}" readonly
                     style="flex:1; padding:6px 8px; border:1px solid #ccc; border-radius:6px;">
              <button onclick="navigator.clipboard.writeText(document.getElementById('dab{idx}').value)"
                      style="padding:6px 10px; border:1px solid #888; border-radius:6px; cursor:pointer;">
                Kopieren
              </button>
            </div>
          </div>

          <div>
            <div style="font-size:12px; color:#555;">Datapoint_ID</div>
            <div style="display:flex; gap:6px; align-items:center;">
              <input id="dp{idx}" type="text" value="{datapoint_id}" readonly
                     style="flex:1; padding:6px 8px; border:1px solid #ccc; border-radius:6px;">
              <button onclick="navigator.clipboard.writeText(document.getElementById('dp{idx}').value)"
                      style="padding:6px 10px; border:1px solid #888; border-radius:6px; cursor:pointer;">
                Kopieren
              </button>
            </div>
          </div>
        </div>
        """

        folium.Marker(
            location=[float(lat), float(lon)],
            popup=folium.Popup(popup_html, max_width=340),
        ).add_to(karte)

# Karte speichern
karte.save("Datenverfügbarkeit Windgeschwindigkeit.html")


mit drei lagen je feature eine:

In [13]:
import pandas as pd
import folium
import math
from folium import FeatureGroup
from folium.map import LayerControl
from branca.element import MacroElement
from jinja2 import Template

# =========================
# Pfade/Sheets und Farben pro Datensatz
# (Passe die Pfade an; wenn alles im selben File aber auf anderen Sheets liegt, setze die Pfade gleich und sheet_name pro Datensatz)
datasets = {
    "Windgeschwindigkeit": {
        "path": r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\Data\Feature Verfügbarkeit\Windgeschwindigkeit.xlsx",
        "sheet_name": None,   # z.B. "Tabelle1"
        "color": "blue"
    },
    "Außentemperatur": {
        "path": r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\Data\Feature Verfügbarkeit\Außentemperatur.xlsx",
        "sheet_name": None,
        "color": "red"
    },
    "Globale Strahlung": {
        "path": r"C:\Users\M97947\OneDrive - E.ON\Dokumente\Thesis\Code\fca_leistungsbaender\Data\Feature Verfügbarkeit\Globale Strahlung.xlsx",
        "sheet_name": None,
        "color": "green"
    },
}
# Erwartete Spalten in jeder Datei/Sheet:
#   Ort, Latitude, Longitude, DAB_ID, Datapoint_ID
# =========================

def safe_str(x):
    if isinstance(x, float) and math.isnan(x):
        return ""
    return str(x) if x is not None else ""

# Karte initialisieren (Schleswig-Holstein)
karte = folium.Map(location=[54.3233, 10.1228], zoom_start=8)

for layer_name, cfg in datasets.items():
    try:
        # Excel-Datei laden
        df_loaded = pd.read_excel(cfg["path"], sheet_name=cfg["sheet_name"])
        # Wenn None → alle Sheets → erstes nehmen
        if isinstance(df_loaded, dict):
            df = next(iter(df_loaded.values()))
        else:
            df = df_loaded
    except Exception as e:
        print(f"Warnung: Konnte '{layer_name}' nicht laden ({cfg['path']}): {e}")
        continue


    fg = FeatureGroup(name=layer_name, show=True)  # Layer standardmäßig sichtbar

    for idx, row in df.iterrows():
        ort = safe_str(row.get('Ort'))
        lat = row.get('Latitude')
        lon = row.get('Longitude')
        dab_id = safe_str(row.get('DAB_ID'))
        datapoint_id = safe_str(row.get('Datapoint_ID'))

        if pd.notna(lat) and pd.notna(lon):
            popup_html = f"""
            <div style="font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial; font-size:14px; line-height:1.3; min-width:260px;">
              <div style="font-weight:600; margin-bottom:6px;">{ort}</div>

              <div style="margin-bottom:6px;">
                <div style="font-size:12px; color:#555;">Datensatz</div>
                <div>{layer_name}</div>
              </div>

              <div style="margin-bottom:6px;">
                <div style="font-size:12px; color:#555;">DAB_ID</div>
                <div style="display:flex; gap:6px; align-items:center;">
                  <input id="dab{layer_name[:3]}{idx}" type="text" value="{dab_id}" readonly
                        style="flex:1; padding:6px 8px; border:1px solid #ccc; border-radius:6px;">
                  <button onclick="navigator.clipboard.writeText(document.getElementById('dab{layer_name[:3]}{idx}').value)"
                          style="padding:6px 10px; border:1px solid #888; border-radius:6px; cursor:pointer;">
                    Kopieren
                  </button>
                </div>
              </div>

              <div>
                <div style="font-size:12px; color:#555;">Datapoint_ID</div>
                <div style="display:flex; gap:6px; align-items:center;">
                  <input id="dp{layer_name[:3]}{idx}" type="text" value="{datapoint_id}" readonly
                        style="flex:1; padding:6px 8px; border:1px solid #ccc; border-radius:6px;">
                  <button onclick="navigator.clipboard.writeText(document.getElementById('dp{layer_name[:3]}{idx}').value)"
                          style="padding:6px 10px; border:1px solid #888; border-radius:6px; cursor:pointer;">
                    Kopieren
                  </button>
                </div>
              </div>
            </div>
            """

            # Marker mit Farb-Icon je Layer
            folium.Marker(
                location=[float(lat), float(lon)],
                popup=folium.Popup(popup_html, max_width=340),
                icon=folium.Icon(color=cfg["color"], icon="info-sign")
            ).add_to(fg)

    fg.add_to(karte)

# Layer-Controller (Checkboxen oben rechts)
LayerControl(collapsed=False).add_to(karte)

# Kleine Farblegende (unten links)
legend_html = """
{% macro html(this, kwargs) %}
<div id='dataset-legend' style="
 position: fixed;
 bottom: 20px;
 left: 20px;
 z-index: 9999;
 background: white;
 padding: 10px 12px;
 border: 1px solid #ccc;
 border-radius: 8px;
 box-shadow: 0 1px 4px rgba(0,0,0,0.2);
 font-family: system-ui, -apple-system, Segoe UI, Roboto, Arial;
 font-size: 13px;">
  <div style="font-weight:600; margin-bottom:6px;">Datensätze</div>
  <div style="display:flex; gap:10px; align-items:center; margin:4px 0;">
    <span style="display:inline-block; width:12px; height:12px; background:#2a81cb; border-radius:3px; border:1px solid #1c5a8a;"></span>
    <span>Windgeschwindigkeit</span>
  </div>
  <div style="display:flex; gap:10px; align-items:center; margin:4px 0;">
    <span style="display:inline-block; width:12px; height:12px; background:#d94c4c; border-radius:3px; border:1px solid #9d2d2d;"></span>
    <span>Außentemperatur</span>
  </div>
  <div style="display:flex; gap:10px; align-items:center; margin:4px 0;">
    <span style="display:inline-block; width:12px; height:12px; background:#3bb273; border-radius:3px; border:1px solid #2a7e52;"></span>
    <span>Globale Strahlung</span>
  </div>
  <div style="margin-top:6px; color:#666;">Hinweis: Sichtbarkeit über die Layer-Control oben rechts.</div>
</div>
{% endmacro %}
"""
class Legend(MacroElement):
    def __init__(self, template):
        super().__init__()
        self._template = Template(template)

karte.get_root().add_child(Legend(legend_html))

# Karte speichern
karte.save("Datenverfuegbarkeit_FeatMap.html")
print("Karte gespeichert: Datenverfuegbarkeit_FeatMap.html")


Karte gespeichert: Datenverfuegbarkeit_FeatMap.html


In [12]:
import os
os.getcwd()

'c:\\Users\\M97947\\OneDrive - E.ON\\Dokumente\\Thesis\\Code\\fca_leistungsbaender\\exploratory'